В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: quantile transformation для числовых признаков и standart scale для категориальных OHE

Добавление шума: ко всем признакам

In [1]:
%cd ..

/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/workspace/TabDDPM_copy/TabDDPM


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ON_QnSC.tabddpm_ON_QnSC import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [3]:
dataname = 'beijing_ON_QnSC_200k'
dataname_initial = 'beijing'
model_short = 'tabddpm_ON_QnSC'
model_name = 'tabddpm_ON_QnSC'
model_name_postfix = '_200k'

idx = list(range(10))
constants = [0, 0.001, 0.01, 0.1, 0.25, 0.5]

CONFIG.add_arg('save_cat', None)
target_cat=False

In [4]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [5]:
model_save_path_hist = []
for sigma in constants:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i in idx:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_mult_{i}"
    model_save_path_hist.append(model_save_path)

In [6]:
from pickle import dump, load
with open(f'data/{dataname}/normalizers.json', 'r') as f:
        normalizers = json.load(f)
normalizers['num_normalizer'] = load(open(f'./data/{dataname}/num_normalizer_{dataname}.pkl', 'rb'))
normalizers['cat_normalizer'] = load(open(f'./data/{dataname}/cat_normalizer_{dataname}.pkl', 'rb'))
print(f"normalizers received from `data/{dataname}` folder")

normalizers received from `data/beijing_ON_QnSC_200k` folder


In [7]:
CONFIG.get_all_args()

{'dataname': 'beijing_ON_QnSC_200k',
 'method': 'tabddpm',
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': 'synthetic/beijing/tabddpm.csv',
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 66,
 'real_path': './synthetic/beijing_ON_QnSC_200k/initial_real.csv',
 'test_path': './synthetic/beijing_ON_QnSC_200k/initial_test.csv',
 'info_path': './data/beijing_ON_QnSC_200k/initial_info.json',
 'save_path': './synthetic/shoppers_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_mult_9.csv',
 'num_clusters': 25,
 'save_cat': None}

### Sample + Eval

In [13]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise_QnSC(CONFIG, model_save_path=model_save_path, sigmas=None)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv', normalizers=normalizers, target_cat=target_cat)

    # подсчет метрик
    # try:
    #     overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=False) # ошибка
    # except Exception as e:
    #     overall_metrics[model_name]['base_metrics'] = np.nan
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

./models/tabddpm_ON_QnSC/ckpt/beijing/tabddpm_ON_QnSC_200k_const_0 tabddpm_ON_QnSC_200k_const_0 ./synthetic/beijing_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Shape torch.Size([37581, 83])
Sampling time: 199.2336893081665
Sample path changed to ./synthetic/beijing_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_const_0.csv
CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 12/12 [00:00<00:00, 41.72it/s]|
Column Shapes Score: 97.99%

(2/2) Evaluating Column Pair Trends: |██████████| 66/66 [00:00<00:00, 81.35it/s]|
Column Pair Trends Score: 96.7%

Overall Score (Average): 97.34%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNSC_200K_CONST_0: 2.011 ± 1.964
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNSC_200K_CONST_0: 3.303 ± 2.502


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:24<00:00,  1.44it/s]


Saving scores to  eval/mle/beijing_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0.json
RMSE обучения на синтетических данных TABDDPM_ON_QNSC_200K_CONST_0: 0.545 ± 0.043
beijing_ON_QnSC_200k, tabddpm_ON_QnSC_200k_const_0: 0.9135241126286776
n_clusters: 25
DCR Score, a value closer to 0.5 is better
beijing_ON_QnSC_200k-tabddpm_ON_QnSC_200k_const_0, DCR Score = 0.5089806018998962
=========== All Features ===========
Data shape:  (37581, 83)
alpha precision: 0.974778, beta recall: 0.560530
./models/tabddpm_ON_QnSC/ckpt/beijing/tabddpm_ON_QnSC_200k_const_0_001 tabddpm_ON_QnSC_200k_const_0_001 ./synthetic/beijing_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0_001.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Shape torch.Size([37581, 83])
Sampling time: 198.41256022453308
Sample path changed to ./synthetic/beijing_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_const_0_001.csv
CALCULATING COLUMN-WISE DENSITY ESTIMATION & PA

...

Saving scores to  eval/mle/beijing_ON_QnSC_200k/tabddpm_ON_QnSC_200k_mult_8.json
RMSE обучения на синтетических данных TABDDPM_ON_QNSC_200K_MULT_8: 0.579 ± 0.041
beijing_ON_QnSC_200k, tabddpm_ON_QnSC_200k_mult_8: 0.8920349787467505
n_clusters: 25
DCR Score, a value closer to 0.5 is better
beijing_ON_QnSC_200k-tabddpm_ON_QnSC_200k_mult_8, DCR Score = 0.5009712354647295
=========== All Features ===========
Data shape:  (37581, 83)
alpha precision: 0.965287, beta recall: 0.575725
./models/tabddpm_ON_QnSC/ckpt/beijing/tabddpm_ON_QnSC_200k_mult_9 tabddpm_ON_QnSC_200k_mult_9 ./synthetic/beijing_ON_QnSC_200k/tabddpm_ON_QnSC_200k_mult_9.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Shape torch.Size([37581, 83])
Sampling time: 197.91942858695984
Sample path changed to ./synthetic/beijing_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_mult_9.csv
CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATI

DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:22<00:00,  1.57it/s]


Saving scores to  eval/mle/beijing_ON_QnSC_200k/tabddpm_ON_QnSC_200k_mult_9.json
RMSE обучения на синтетических данных TABDDPM_ON_QNSC_200K_MULT_9: 0.556 ± 0.044
beijing_ON_QnSC_200k, tabddpm_ON_QnSC_200k_mult_9: 0.9002641030692083
n_clusters: 25
DCR Score, a value closer to 0.5 is better
beijing_ON_QnSC_200k-tabddpm_ON_QnSC_200k_mult_9, DCR Score = 0.5059205449562278
=========== All Features ===========
Data shape:  (37581, 83)
alpha precision: 0.963272, beta recall: 0.586571


In [14]:
final_metrics_table = []
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':model_short, 'Type':m, 'Data':dataname_initial}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [15]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(4).sort_values([('similarity', 'Overall Score (Average), %')],
                                                                          ascending=False)

\
             Model                              Type     Data   
0  tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_const_0  beijing   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_25  beijing   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_4  beijing   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_9  beijing   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_6  beijing   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_1  beijing   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_8  beijing   
0  tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_001  beijing   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_5  beijing   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_01  beijing   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_0  beijing   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_1  beijing   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_5  beijing   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_3  beijing   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_7  beijing   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_2  beijing   

              similarity                              \
  Column Shapes Score, % Column Pair Trends Score, %   
0                97.9895                     96.6970   
0                98.0790                     96.5794   
0                97.6839                     96.6905   
0                97.6493                     96.6649   
0                97.7737                     96.2994   
0                97.7497                     96.1751   
0                97.6668                     96.1065   
0                97.6777                     96.0761   
0                98.0340                     95.4794   
0                97.4806                     95.6366   
0                96.9774                     95.2331   
0                96.9118                     95.0557   
0                97.0255                     94.7850   
0                96.4816                     94.4061   
0                96.8395                     93.9810   
0                96.3523                     93.9562   

                              \
  Overall Score (Average), %   
0                    97.3432   
0                    97.3292   
0                    97.1872   
0                    97.1571   
0                    97.0366   
0                    96.9624   
0                    96.8867   
0                    96.8769   
0                    96.7567   
0                    96.5586   
0                    96.1053   
0                    95.9837   
0                    95.9053   
0                    95.4439   
0                    95.4102   
0                    95.1542   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                              2.0105   
0                                              1.9210   
0                                              2.3161   
0                                              2.3507   
0                                              2.2263   
0                                              2.2503   
0                                              2.3332   
0                                              2.3223   
0                                              1.9660   
0                                              2.5194   
0                                              3.0226   
0                                              3.0882   
0                                              2.9745   
0                                              3.5184   
0                                              3.1605   
0                                              3.6477   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                  1.9636   
0                                                  1.5076   
0                                  

In [16]:
import os
if not os.path.exists(f'./eval/total/{dataname_initial}'):
    os.makedirs(f'./eval/total/{dataname_initial}')
final_metrics_table.to_csv(f'./eval/total/{dataname_initial}/{model_short}{model_name_postfix}_final_metrics_table.csv', index=False)